[1] 필요한 패키지 불러오기

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris

[2] 데이터 준비 함수 정의하기

In [ ]:
def prepare_data(target) :
    iris = load_iris() 
    X_tr = iris.data[:, 2:] # 4개의 특징 중 꽃잎의 길이와 폭 선택
    labels = iris.target_names # iris data가 sklearn.datasets.data 형태여서 target_naems라는 것이 정의되어있음
    y = iris.target

    # 학습 표본의 레이블 지정 - target에 지정된 레이블이면 1, 그 외는 0
    y_tr = []
    for i in range(150) : # iris data 갯수 150
            y_tr.append(labels[y[i]] == target) 
        y_tr = np.array(y_tr, dtype=int)
        return X_tr, y_tr ['(1) '+target, '(0) the others']

[3] 활성함수 - 시그모이드

In [ ]:
def sigmoid(x) :
    ''' x : numpy array '''
    return 1 / (1 + np.exp(-x))


[4] 손실함수 - mse,corss entropy

In [ ]:
def loss_mse(y, y_hat) :
    loss = 0.0
    for i in range(len(y)):
        err = y_hat[i] - y[i]
        loss += np.dot(err,err)
    return loss / len(y) #평균제곱오차

def loss_ce(y, y_hat):
    loss = 0.0
    if len(y.shape) == 1 or y.shape[1] == 1: #y가 1차원배열 or 두번째차원 크기가 1 Binary cross-entropy case
        for i in range(len(y)) :
            loss += -(y[i] * np.log(y_hat[i])
                      + (1-y[i]) * np.log((1-y_hat[i]))).sum()
    else : # Categorical cross-entropy case
        for i in range(len(y)):
            loss += -(y[i] * np.log(y_hat[i])).sum()
    return loss / len(y)

[5] Dense 클래스 - 완전연결층

In [ ]:
class Dense():
    def __init__(self, nIn, nOut, activation='sigmoid', loss='mse'):
        self.nIn = nIn # 입력의 수
        self.nOut = nOut # 출력의 수
        # 가중치(w)와 바이어스(b)를 He normal 방식으로 초기화
        rnd = np.random.default_rng()
        self.w == rnd.normal(scale = np.sqrt(2.0/ self.nIn),
                             size = (self.nOut, self.nIn)).astype(np.float32)
        self.b == rnd.normal(scale = np.sqrt(2.0/ self.nIn),
                             size = self.nOut).astype(np.float32)
        # 활성함수 설정
        if activation == 'sigmoid':
            self.activation = sigmoid
            if loss =='ce':    self.dE_du = self.dE_du_sigmoid_ce
            else :              self.dE_du = self.dE_du_sigmoid_mse
            self.do_du = self.do_du_sigmoid
        # 모멘텀을 적용하기 위한 속도의 초깃값 설정
        self.velocity_w, self.velocity_b = 0.0, 0.0
    
    # 입력 X에 대한 출력 계산
    def output(self,X):
        self.in_vec = X bp 학습을 위해 입력 보관
        # 입력의 가중 합 계산
        u = np.array([np.dot(self.w[i], X) + self.b[i]
                      for i in range(self.nOut)], dtype=np.float32)
        # 활성함수를 적용한 출력 계산
        self.out_vec = self.activation(u) #bp 학습을 위해 출력 보관
        return self.out_vec
    
    # 경사 하강법에 따라 w 및 b 갱신
    def gd(self, dw, db, momentum=0):
        self.velocity_w = self.velocity_w * momentum -dw
        self.velocity_b = self.velocity_b * momentum -db
        self.w += self.velocity_w
        self.b += self.velocity_b

    def dE_du_sigmoid_mse(self, y):
        return (self.out_vec - y)
    
    def do_du_sigmoid(self):
        return self.out_vec * (1-self.out_vec)